In [ ]:
import numpy as np
import os
from scipy.io import wavfile
import librosa
import csv
import math as m
from natsort import natsorted
import h5py
#in this section the features for mixed voices are extracted.
# orig_path=os.getcwd()
i = 1
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
mixed_folder = os.path.normpath(os.path.join(Data_path,'mixed10h'))
# f.close()
a = os.listdir(os.path.join(mixed_folder))
a = natsorted(a)
mixed_logname = 'mixed_log_10h'
mixed_phasename = 'mixed_phase_10h'
mixed_logpath = os.path.normpath(os.path.join(write_path,mixed_logname))
mixed_phasepath = os.path.normpath(os.path.join(write_path,mixed_phasename))

f = h5py.File(mixed_logpath+'.hdf5', 'w')
d = f.create_dataset(mixed_logname, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# ff.close()
ff = h5py.File(mixed_phasepath+'.hdf5', 'w')
dd = ff.create_dataset(mixed_phasename, (0,257),maxshape=(None,257), dtype='f', chunks=True)
#################################
w=3
#################################
for filename in a:
    wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
#     print(framed_data.shape)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
    for i in range(m.floor(w/2)):
        dft_signal=np.insert(dft_signal,0,0,axis = 1)
    data_phase=np.angle(framed_data)
    for i in range(m.floor(w/2)):
        data_phase=np.insert(data_phase,0,0,axis = 1)
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T
    i=i+1
####################################################
####################################################
import h5py
import math as m
import os
import numpy as np
import librosa
from natsort import natsorted
orig_path=os.getcwd()
wav=[]
clean_raw = 'clean_10h'
# l=os.listdir(os.path.normpath(os.path.join(Data_path,clean_raw)))
# l=natsorted(l)
# filename1 = l[1]
i=0
###################################################
w=3
###################################################
num_of_snrs=6
num_of_speakers=2
files = os.listdir(os.path.join(Data_path,clean_raw))
files=natsorted(files)
# f.close()
# ff.close()
clean_log = 'single_dataset_log_10h_target_norm'
clean_phase = 'single_dataset_phase_10h_target_norm'
clean_log_path = os.path.normpath(os.path.join(write_path,clean_log))
clean_phase_path = os.path.normpath(os.path.join(write_path,clean_phase))
f = h5py.File(clean_log_path+'.hdf5', 'w')
d = f.create_dataset(clean_log, (0,257),maxshape=(None,257), dtype='f', chunks=True)
ff = h5py.File(clean_phase_path+'.hdf5', 'w')
dd = ff.create_dataset(clean_phase, (0,257),maxshape=(None,257), dtype='f', chunks=True)
#in this section the features for clean voices are extracted.
for voice in files:
    wav_data, sr = librosa.load(os.path.join(Data_path,clean_raw,voice), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
#     print(framed_data.shape)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
    data_phase=np.angle(framed_data)
#     for i in range(m.floor(w/2)):
#         dft_signal=np.insert(dft_signal,0,0,axis = 1)
#     for i in range(num_of_snrs*num_of_speakers):
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T
f.close()
ff.close()
###############################################
###############################################
import math as m
import numpy as np
import h5py
##############################################
w = 3
###############################################
mixed_logname='mixed_log_10h'
mixed_phasename='mixed_phase_10h'
mixed_log_path = os.path.normpath(os.path.join(write_path,mixed_logname))
clean_phase_path = os.path.normpath(os.path.join(write_path,mixed_phasename))
import h5py
h5f = h5py.File(mixed_log_path+'.hdf5','r')
ftr = h5f[mixed_logname][0:]
# h5f.close()
ftr_refrmd=[]
print('0')
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
f.close()
print('1')
refrmd_file='ftr_refrmd_10h'
refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
f = h5py.File(refrmd_file_path+'.hdf5', 'w')
d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
print('2')
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>50000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]
f.close()
h5f.close()
##############################################
##############################################
# import h5py
# import numpy as np
# h5 = h5py.File(refrmd_file, 'r')
# d=h5[refrmd_file]
# tot_len=d.shape[0]
# lst=0
# h5.close()
# on=0
# h5f = h5py.File(refrmd_file+'.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f[refrmd_file][i:i+1000]
#     lst=lst+np.sum(mixed,axis=0)
#     on=on+len(mixed)
# tot_mean=lst/tot_len
# h5f.close()
# lst=0
# h5f = h5py.File(refrmd_file+'.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f[refrmd_file][i:i+1000]
#     lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
# lst=lst/tot_len
# tot_std=np.sqrt(lst)
# h5f.close()
# f = h5py.File('ftr_scaled.hdf5', 'w')
# dd = f.create_dataset('ftr_scaled', (0,1799),maxshape=(None,1799), dtype='f', chunks=True)
# h5f = h5py.File('ftr_refrmd.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f['ftr_refrmd'][i:i+1000]
#     scaled=(mixed-tot_mean)/tot_std
#     dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
#     dd[-1*scaled.shape[0]:] = scaled
# np.savetxt('mean.txt',tot_mean)
# np.savetxt('std.txt',tot_std)

In [ ]:
import numpy as np
import tensorflow as tf
import os
__author__ = "Sangwoong Yoon"

def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print("Writing {} done!".format(result_tf_file))

        
#################################    
##      Test and Use Cases     ##
#################################
import h5py
load_size = 500000
k = 0
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
input_name = 'ftr_refrmd_10h'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'single_dataset_log_10h_target_norm'
target_path = os.path.normpath(os.path.join(write_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
index = np.arange(0, data_len, load_size)
for I in index:
    if data_len-I<load_size:
        xx = h5f1[input_name][I:]
        yy = hh[target_name][I:]
    else:
        xx = h5f1[input_name][I:I+load_size]
        yy = hh[target_name][I:I+load_size]
    np_to_tfrecords(xx, yy, os.path.normpath(os.path.join(Data_path,'tfrecord_files','tfrecord_files_10h','filenum'+str(k))), verbose=True)
    k = k+1
# 1-2. Check if the data is stored correctly
# open the saved file and check the first entries
# for serialized_example in tf.python_io.tf_record_iterator('test.tfrecords'):
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     x_1 = np.array(example.features.feature['X'].float_list.value)
#     y_1 = np.array(example.features.feature['Y'].float_list.value)
#     break

# np_to_tfrecords(X, Y, file_path_prefix, verbose=True)